# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [2]:
pip install "dask[dataframe]" --upgrade

  Using cached pandas-2.2.2-cp39-cp39-win_amd64.whl.metadata (19 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/206.6 kB ? eta -:--:--
   ---------------------------------------- 0.0/206.6 kB ? eta -:--:--
   - -------------------------------------- 10.2/206.6 kB ? eta -:--:--
   - -------------------------------------- 10.2/206.6 kB ? eta -:--:--
   ----------- --------------------------- 61.4/206.6 kB 465.5 kB/s eta 0:00:01
   -------------------------- ----------- 143.4/206.6 kB 944.1 kB/s eta 0:00:01
   ------------------------------ ------- 163.8/206.6 kB 756.6 kB/s eta 0:00:01
   -------------------------------------- 206.6/206.6 kB 896.7 kB/s eta 0:00:00
Using cached pandas-2.2.2-cp39-cp39-win_amd64.whl (11.6 MB)
Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled p

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
islp 0.3.22 requires pandas<=1.9,>=0.20, but you have pandas 2.2.2 which is incompatible.


In [6]:
pip install dask

Note: you may need to restart the kernel to use updated packages.


In [1]:
# Write your code below.
%reload_ext dotenv 
%dotenv 
# -o "02_activities\assignments\.env"



In [156]:
import os 
os.getenv('PRICE_DATA')
# print(os.environ)


'../../05_src/data/prices/'

In [2]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob

# Write your code below.

#price_data directory from .env
PRICE_DATA = os.getenv("PRICE_DATA")

#path of all parquet files 
parquet_files = glob(os.path.join(PRICE_DATA, "*/*.parquet"))
dd_px = dd.read_parquet(parquet_files).set_index("ticker")


In [13]:
dd_px.compute()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,year
ticker,,,,,,,,,
A,2013-12-02 00:00:00-05:00,34.931002,35.061655,34.695820,34.761147,2039962,0.0,0.0,2013
A,2013-12-03 00:00:00-05:00,34.571688,34.735011,34.336506,34.578220,3462706,0.0,0.0,2013
A,2013-12-04 00:00:00-05:00,34.519432,35.166184,34.441041,35.002861,3377288,0.0,0.0,2013
A,2013-12-05 00:00:00-05:00,34.852604,35.146580,34.702349,34.950596,2530939,0.0,0.0,2013
A,2013-12-06 00:00:00-05:00,35.120441,35.884777,35.120441,35.819450,4268513,0.0,0.0,2013
...,...,...,...,...,...,...,...,...,...
YUM,2024-01-25 00:00:00-05:00,128.551783,129.027053,127.274475,128.937943,1871700,0.0,0.0,2024
YUM,2024-01-26 00:00:00-05:00,128.640891,129.403315,127.403198,127.819061,1157000,0.0,0.0,2024
YUM,2024-01-29 00:00:00-05:00,127.828973,129.264694,127.026935,129.264694,1482100,0.0,0.0,2024


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [16]:
import numpy as np
dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1))
).assign(
    Adj_Close = lambda x: x['Close'] 
    #assuming that the adjusted close is same as daily close becuase of unavailability of adjusted close data. 
).assign(
    Adj_Close_lag_1 = lambda x: x['Adj_Close'].shift(1) 
    #assuming that the adjusted close is same as daily close becuase of unavailability of adjusted close data. 
).assign(
    returns = lambda x: x['Adj_Close']/x['Adj_Close_lag_1'] - 1
).assign(
    hi_lo_range = lambda x: (x['High'] - x['Low'] )
))

C:\Users\engr_\AppData\Local\Temp\ipykernel_1928\2386241124.py:2: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(


In [19]:
dd_feat.compute()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,year,Close_lag_1,Adj_Close,Adj_Close_lag_1,returns,hi_lo_range
ticker,,,,,,,,,,,,,,
A,2013-12-02 00:00:00-05:00,34.931002,35.061655,34.695820,34.761147,2039962,0.0,0.0,2013,NaN,34.761147,NaN,NaN,0.365836
A,2013-12-03 00:00:00-05:00,34.571688,34.735011,34.336506,34.578220,3462706,0.0,0.0,2013,34.761147,34.578220,34.761147,-0.005262,0.398505
A,2013-12-04 00:00:00-05:00,34.519432,35.166184,34.441041,35.002861,3377288,0.0,0.0,2013,34.578220,35.002861,34.578220,0.012281,0.725143
A,2013-12-05 00:00:00-05:00,34.852604,35.146580,34.702349,34.950596,2530939,0.0,0.0,2013,35.002861,34.950596,35.002861,-0.001493,0.444231
A,2013-12-06 00:00:00-05:00,35.120441,35.884777,35.120441,35.819450,4268513,0.0,0.0,2013,34.950596,35.819450,34.950596,0.024860,0.764337
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YUM,2024-01-25 00:00:00-05:00,128.551783,129.027053,127.274475,128.937943,1871700,0.0,0.0,2024,128.165634,128.937943,128.165634,0.006026,1.752578
YUM,2024-01-26 00:00:00-05:00,128.640891,129.403315,127.403198,127.819061,1157000,0.0,0.0,2024,128.937943,127.819061,128.937943,-0.008678,2.000117
YUM,2024-01-29 00:00:00-05:00,127.828973,129.264694,127.026935,129.264694,1482100,0.0,0.0,2024,127.819061,129.264694,127.819061,0.011310,2.237759


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [32]:
# Write your code below.

#Saving Dask dataframe in panda dataframe 
pd_feat = dd_feat.compute()
#validating the panda dataframe 
pd_feat.describe()

# Applying rolling average on panda dataframe 

# Assuming that the rolling avg requirement is to be based on a particular ticker 
pd_feat['moving'] = pd_feat.groupby('ticker')['returns'].transform(lambda x: x.rolling(10, 1).mean())
    # If 10 returns are not present, the average will be based on minimum 1 return. 



In [33]:
pd_feat.head(10)

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,year,Close_lag_1,Adj_Close,Adj_Close_lag_1,returns,hi_lo_range,moving
ticker,,,,,,,,,,,,,,,
A,2013-12-02 00:00:00-05:00,34.931002,35.061655,34.695820,34.761147,2039962,0.0,0.0,2013,NaN,34.761147,NaN,NaN,0.365836,NaN
A,2013-12-03 00:00:00-05:00,34.571688,34.735011,34.336506,34.578220,3462706,0.0,0.0,2013,34.761147,34.578220,34.761147,-0.005262,0.398505,-0.005262
A,2013-12-04 00:00:00-05:00,34.519432,35.166184,34.441041,35.002861,3377288,0.0,0.0,2013,34.578220,35.002861,34.578220,0.012281,0.725143,0.003509
A,2013-12-05 00:00:00-05:00,34.852604,35.146580,34.702349,34.950596,2530939,0.0,0.0,2013,35.002861,34.950596,35.002861,-0.001493,0.444231,0.001842
A,2013-12-06 00:00:00-05:00,35.120441,35.884777,35.120441,35.819450,4268513,0.0,0.0,2013,34.950596,35.819450,34.950596,0.024860,0.764337,0.007596
A,2013-12-09 00:00:00-05:00,35.799864,36.211429,35.760666,35.969715,2961663,0.0,0.0,2013,35.819450,35.969715,35.819450,0.004195,0.450763,0.006916
A,2013-12-10 00:00:00-05:00,35.871726,36.231033,35.688806,36.152637,3253006,0.0,0.0,2013,35.969715,36.152637,35.969715,0.005085,0.542227,0.006611
A,2013-12-11 00:00:00-05:00,36.159182,36.381299,35.721483,35.826008,3277751,0.0,0.0,2013,36.152637,35.826008,36.152637,-0.009035,0.659816,0.004376
A,2013-12-12 00:00:00-05:00,35.688807,36.270231,35.473225,36.100376,3794591,0.0,0.0,2013,35.826008,36.100376,35.826008,0.007658,0.797006,0.004786


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

It was not necessary to convert to pandas for calculating rolling averages. Dask is prefered over pandas when we want to utilize the power of parallel processing whereas pandas is better choice for sequential tasks. 

Dask can perform rolling average without any problem. Dask can perform the same operation much faster by utilizing multi-thread/ multi-core processing. 

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.